In [1]:
from sage.rings.polynomial.pbori.pbori import BooleSet
import json
import itertools
import pandas as pd
import numpy as np

# Calculating Trace

Our goal is to produce a polynomial function which, when fed the binary representation of a group element, produces the trace of that element. For our chosen group representation of $\mathbb{2O}$, the binary octahedral group, there are 8 possible traces. Namely $\pm{0},\pm{1},\pm{2}$, and $\pm{\sqrt{2}}$, which we shall denote as $T$ for now. 

Each element of a finite collection $X$ can be described by some map $\mathbb{Z_2^{|X|}} \rightarrow X$, which assigns a binary "address" to each element of $X$. In particular we have that $\mathbb{2O}$ (the binary octahedral group) is addressed by $\mathbb{Z_2^6}$, and our set of traces by $\mathbb{Z_2^3}$. Given a choice of address functions 
$$\text{group_addr} \colon \mathbb{Z_2^6} \rightarrow \mathbb{2O}$$
$$\text{trace_addr} \colon \mathbb{Z_2^3} \rightarrow T$$
and the trace map $$\text{Tr} \colon \mathbb{2O} \rightarrow T$$
we can rephrase our goal as finding a pullback map $f \colon \mathbb{Z_2^6} \rightarrow \mathbb{Z_2^3}$ such that
$$\text{Tr} \circ \text{group_addr} = \text{trace_addr} \circ f$$

To do this we start by defining our two rings of functions. The way theyre instatiated is purely for computation purposes and not important.

In [2]:
Z2_6.<w1,w2,w3,w4,w5,w6> = BooleanPolynomialRing(6)
R_3.<v1,v2,v3> = PolynomialRing(RR)

Here we define out trace address function

In [3]:
canonical_trace_poly = 1*v1*(1-v2)*v3 - 1*v1*(1-v2)*(1-v3) + sqrt(2)*(1-v1)*v2*v3 - sqrt(2)*(1-v1)*v2*(1-v3) + 2*(1-v1)*(1-v2)*v3 - 2*(1-v1)*(1-v2)*(1-v3)

Here we define out matrix elements directly. We will omit the actual map $\mathbb{Z_2^6} \rightarrow \mathbb{2O}$, as it ends up being implicit in how we construct our dataset later on.

In [4]:
m1 = Matrix([[1,0],
             [0,1]])

mi = Matrix([[0, 1j],
             [1j,0]])

mj = Matrix([[0,1],
             [-1,0]])

mk = Matrix([[1j, 0],
             [0, -1j]])

mu = 1/2 * ((-m1) + mi + mj + mk)

mt = 1/sqrt(2) * (m1 + mi)

We produce a table of inputs and outputs for our trace_address function. We also specifically omit the last case, as the solver has trouble with 0 occurring twice.

In [5]:
traces = pd.DataFrame([{"trace" : canonical_trace_poly(v1=p1,v2=p2,v3=p3),
                        "v1" : p1,
                        "v2" : p2,
                        "v3" : p3 } for p1, p2, p3 in itertools.product(range(2),range(2),range(2))])
#traces = traces[traces.v2 + traces.v3 != 2]

Here we produce the table of elements alongside their various representations, trace, and standard address digits. The map to the reprsentation from the address space is implicit here, as we construct those elements via the same iterator.

In [6]:
trace_table = pd.DataFrame([{ "melement" : (-m1)^p1 * mj^p2 * mk^p3 * mu^(2 * p4 + p5) * mt^p6,
                              "trace" : 0 if (p4 == 1 and p5 == 1) else ((-m1)^p1 * mj^p2 * mk^p3 * mu^(2 * p4 + p5) * mt^p6).trace(),
                              "w1" : p1,
                              "w2" : p2,
                              "w3" : p3,
                              "w4" : p4,
                              "w5" : p5,
                              "w6" : p6 }
                                              for p1, p2, p3, p4, p5, p6
                                              in itertools.product(range(2),range(2),range(2),range(2),range(2),range(2))])

#trace_table = trace_table[(trace_table.w4 + trace_table.w5) != 2]

Now we join the tables across the common trace, so that we can associate the 6 bit standard form to the 3 bit trace form. This gives the data of the map $f$ that we are after.

In [7]:
trace_table_idens = traces.join(trace_table.set_index("trace"), on = "trace")

We generate polynomials mapping the standard bit representation to the correspondong trace representation. Each one one component of $f$.

In [8]:
trace_poly_v1_zeros = trace_table_idens[trace_table_idens.v1 == 0].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v1_zeros = set(list(map(tuple, trace_poly_v1_zeros)))
trace_poly_v1_ones = trace_table_idens[trace_table_idens.v1 == 1].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v1_ones = set(list(map(tuple, trace_poly_v1_ones)))

trace_poly_v2_zeros = trace_table_idens[trace_table_idens.v2 == 0].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v2_zeros = set(list(map(tuple, trace_poly_v2_zeros)))
trace_poly_v2_ones = trace_table_idens[trace_table_idens.v2 == 1].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v2_ones = set(list(map(tuple, trace_poly_v2_ones)))

trace_poly_v3_zeros = trace_table_idens[trace_table_idens.v3 == 0].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v3_zeros = set(list(map(tuple, trace_poly_v3_zeros)))
trace_poly_v3_ones = trace_table_idens[trace_table_idens.v3 == 1].loc[:,["w1","w2","w3","w4","w5","w6"]].values
trace_poly_v3_ones = set(list(map(tuple, trace_poly_v3_ones)))

In [ ]:
trace_poly_v1 = Z2_6.interpolation_polynomial(trace_poly_v1_zeros, trace_poly_v1_ones)
trace_poly_v2 = Z2_6.interpolation_polynomial(trace_poly_v2_zeros, trace_poly_v2_ones)
trace_poly_v3 = Z2_6.interpolation_polynomial(trace_poly_v3_zeros, trace_poly_v3_ones)

Finally we put it all togeth in a test. Each component function $v_i$ when fed an address, should give a component of the address according to our chosen trace address. So with all 3, we should be able to pick it out.

In [ ]:
results = []
for p1, p2, p3, p4, p5, p6 in list(itertools.product(range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2),
                                                     range(2))):
    #Ignore impossible case (where our polys fail!)
    
    tmp1 = RR(trace_poly_v1(p1,p2,p3,p4,p5,p6))
    tmp2 = RR(trace_poly_v2(p1,p2,p3,p4,p5,p6))
    tmp3 = RR(trace_poly_v3(p1,p2,p3,p4,p5,p6))
    
    if (p4 and p5):
        continue
    test = canonical_trace_poly(v1=tmp1,v2=tmp2,v3=tmp3)
        
    ground_mat = ((-m1)^p1 * mj^p2 * mk^p3 * mu^(2 * p4 + p5) * mt^p6)
    ground = ground_mat.trace()
    results.append([ground_mat,ground,test])

print(all(results))

In [ ]:
results[11]

In [ ]:
trace_table_idens

In [ ]:
canonical_trace_poly(v1=0,v2=1,v3=0)

In [ ]:
sum(trace_table.trace.values)

In [ ]:
a=0.3017767
b=0.0517767
d=0.125
e=0.1767767
f=0.4267767
g=0.0732233

In [ ]:
1*sqrt(2).n()/3

In [ ]:
f-g

In [ ]:
 sqrt(2).n()/16 + 1/8

In [ ]:
a+b+e

In [ ]:
1/8*(1*sqrt(2).n() -1 ) -b